In [159]:
import requests
import json
from bs4 import BeautifulSoup
from datetime import date
import time
import logging
import copy

In [60]:
# fetching new data
today = date.today()
d1 = today.strftime("%y%m%d")
# print(d1)
outdata = requests.get('https://www.nndc.bnl.gov/nudat3/data/output.json')
outdatajson = json.loads(outdata.text)
with open('nudat_indent_'+d1+'.json', 'w') as f:
    f.write(json.dumps(outdatajson, indent=2))

In [62]:
#Time units according to https://www.nndc.bnl.gov/ensdf/ensdf-manual.pdf
time_factors = {'as':0.000000000000000001,'fs':1e-15,
               'ps':1e-12, 'ns':0.000000001,'us':0.000001,'ms': 0.001,
               's':1.,'m' : 60.,'h': 3600.,'d' : 86400.,'y':31536000., 'ky' : 31536000000}

In [132]:
def extract_pxn(nuc):
    outdata = requests.get('https://www.nndc.bnl.gov/nudat3/getdataset.jsp?nucleus='+nuc+'&unc=STD')
    soup = BeautifulSoup(outdata.text, 'html.parser')
    t12data = {"value":None,"unit":None,"uncertaintySTD":None}
    p1ndata = {"value":None,"uncertaintySTD":None}
    p2ndata = {"value":None,"uncertaintySTD":None}
    p3ndata = {"value":None,"uncertaintySTD":None}
#     print(outdata.text)
    capturedata = soup.find_all("td", class_="cellc t12")
    if (len(capturedata)>0):
        first_lvl_data = capturedata[0]
        t12pncontent = first_lvl_data.text
        t12pncontent = [t.strip() for t in t12pncontent.split("%")]
        t12datastr = t12pncontent[0].split()
        if (len(t12datastr)>0):
            if (t12datastr[0]!=">" and t12datastr[0]!="<"):
                t12data = {"value":t12datastr[0],"unit":t12datastr[1],"uncertaintySTD":t12datastr[2]}
        for i in t12pncontent:
            if (i.find('β-n')>-1):
                spldata = i.split()
                if (len(spldata)==4):
                    p1ndata["value"] = float(spldata[2])
                    p1ndata["uncertaintySTD"] = spldata[3]
            if (i.find('β-2n')>-1):
                spldata = i.split()
                if (len(spldata)==4):
                    p2ndata["value"] = float(spldata[2])
                    p2ndata["uncertaintySTD"] = spldata[3]
            if (i.find('β-3n')>-1):
                spldata = i.split()
                if (len(spldata)==4):
                    p3ndata["value"] = float(spldata[2])
                    p3ndata["uncertaintySTD"] = spldata[3]
    return t12data,p1ndata,p2ndata,p3ndata
t12data,p1ndata,p2ndata,p3ndata=extract_pxn('6H')
print(t12data,p1ndata,p2ndata)

{'value': '1.55', 'unit': 'MeV', 'uncertaintySTD': '0.44'} {'value': None, 'uncertaintySTD': None} {'value': None, 'uncertaintySTD': None}


In [134]:
with open('nudat_indent_231116.json','r') as f:
    nudat3data = json.load(f)

open('convert.log', 'w').close()
# print(len(nudat3data["nuclides"]))
dataout = []
for idx,i in enumerate(nudat3data["nuclides"]):
    datum = copy.deepcopy(i)
    
    qValues = i.get("qValues", None)
    Qbn = None
    if (qValues!=None):
        Qbn = qValues.get("betaMinusOneNeutronEmission",None)
    if (Qbn!=None):
        if (float(Qbn['value'])>0):
            time.sleep(2)
            with open('convert.log','a') as f:
                print(idx,i["name"], file=f)
            t12data,p1ndata,p2ndata,p3ndata=extract_pxn(i["name"])
            with open('convert.log','a') as f:
                print(i["name"],t12data,p1ndata,p2ndata,p3ndata, file=f)
            datum["t12_web"] = t12data
            datum["p1ndata_web"] = p1ndata
            datum["p2ndata_web"] = p2ndata
            datum["p3ndata_web"] = p3ndata
    dataout.append(datum)

In [161]:
import numpy as np
np.save("nudat3.npy",dataout)

In [162]:
# import pandas as pd
# df = pd.DataFrame(columns=['name','z','n','a','t12','dt12','p1n','dp1n','p2n','dp2n','p3n','dp3n'])
# for idx,i in enumerate(dataout):
#     if (i.get("levels", None)!=None):
#         if (i.get("p1ndata_web", None)!=None):
#             if (i["p1ndata_web"]["value"]!=None):
#                 print(i["name"])
#                 dat = {'name':i['name'],'z':i['z'],'n':i['n'],'a':i['a'],
#                       't12':i["levels"][0]["halflife"]["inSeconds"]["value"],
#                        'dt12':i["levels"][0]["halflife"]["inSeconds"]["uncertainty"]}
#                 df.loc[i['name']] = dat
#     else:
#         print(i['name'])

In [163]:
# print(df)